# Notebook to compute NOX contributions from traffic
## Maps are show current and forecast maps on an hourly basis


In [1]:

import warnings
import os
from src.green_pont import load_json_with_comments
from src.urban_aq import read_cfd_nox, save_local_aq, scale_cfd_nox
from src.urban_wind import get_wind_extreme_dt, get_wind_extreme_dt_ondemand

import matplotlib.pyplot as plt
import numpy as np

cf=load_json_with_comments('etc/settings_air_quality.json')
    
path_cfd=cf['path_cfd'] #'/projects/urbanair/DATA/DATA_AntwerpWindStudy/wind_ratios'
angles=cf["angles"]
height=cf["height"]
path = cf['path_meteo'] #'data/meteo_input/'
grib_file= cf['grib_file'] # 'GRIBPFDEOD+$hour$h00m00s'
output_path=cf["output_path"]




In [ ]:
xc=153400
yc=211644
L=1500


crop_bounds =(xc-L, yc-L, xc+L, yc+L)
cfd_ratio=read_cfd_nox(path_cfd,angles, height, crop_bounds)
print("Done loading database of cfd results")


In [3]:
LOCATION = ((51.213642, 4.415)) #  center of antwerp to get meteo data from ecmwf
wind_meteo=get_wind_extreme_dt(LOCATION,date='-1')
print(wind_meteo)

2025-11-07 14:30:59 - INFO - Key read from /home/jovyan/.polytopeapirc
2025-11-07 14:30:59 - INFO - Sending request...
{'request': 'class: d1\n'
            'dataset: extremes-dt\n'
            "date: '-1'\n"
            "expver: '0001'\n"
            'feature:\n'
            '  axes: step\n'
            '  points:\n'
            '  - - 51.213642\n'
            '    - 4.415\n'
            '  range:\n'
            '    end: 24\n'
            '    start: 0\n'
            '  type: timeseries\n'
            'levtype: sfc\n'
            'param: 166/165\n'
            'stream: oper\n'
            "time: '0000'\n"
            'type: fc\n',
 'verb': 'retrieve'}
2025-11-07 14:30:59 - INFO - Polytope user key found in session cache for user jovyan
2025-11-07 14:31:01 - INFO - Request accepted. Please poll ./773c36c0-4664-4849-b535-f4200e1f3828 for status
2025-11-07 14:31:01 - INFO - Polytope user key found in session cache for user jovyan
2025-11-07 14:31:01 - INFO - Checking request status (773

                     wind_speed    wind_dir
2025-11-06 00:00:00    2.469537  149.188983
2025-11-06 01:00:00    3.175604  148.667587
2025-11-06 02:00:00    4.644190  141.107570
2025-11-06 03:00:00    4.132827  142.999995
2025-11-06 04:00:00    3.546937  154.593493
2025-11-06 05:00:00    3.341535  146.824053
2025-11-06 06:00:00    3.303536  134.526467
2025-11-06 07:00:00    3.348026  146.107990
2025-11-06 08:00:00    3.426398  150.197841
2025-11-06 09:00:00    3.429693  145.455940
2025-11-06 10:00:00    3.963333  153.806285
2025-11-06 11:00:00    3.596903  168.209773
2025-11-06 12:00:00    3.460886  177.957090
2025-11-06 13:00:00    3.468049  177.739403
2025-11-06 14:00:00    3.547726  182.978527
2025-11-06 15:00:00    2.684146  182.390896
2025-11-06 16:00:00    2.217205  180.806852
2025-11-06 17:00:00    2.489878  188.923193
2025-11-06 18:00:00    2.667902  189.280003
2025-11-06 19:00:00    2.568080  196.496904
2025-11-06 20:00:00    2.226233  185.791049
2025-11-06 21:00:00    2.601870 

In [8]:

# scale meso-scale wind to local urban scale using CFD ratios
nox_local=scale_cfd_nox(wind_meteo, cfd_ratio)
print("Done processing NOX maps")

Scaling  NOx to local urban scale using CFD ratios
             0
hour          
0     0.226541
1     0.155558
2     0.121866
3     0.130746
4     0.203857
5     0.449023
6     0.850661
7     1.301599
8     1.581160
9     1.488191
10    1.541592
11    1.608569
12    1.588962
13    1.553126
14    1.586284
15    1.672183
16    1.757533
17    1.720983
18    1.461503
19    1.030787
20    0.677284
21    0.519936
22    0.440332
23    0.331725
0
[0.22654118]
1
[0.15555812]
2
[0.12186637]
3
[0.13074564]
4
[0.2038566]
5
[0.4490227]
6
[0.85066139]
7
[1.30159907]
8
[1.58115971]
9
[1.48819054]
10
[1.54159184]
11
[1.60856879]
12
[1.58896237]
13
[1.55312617]
14
[1.58628379]
15
[1.67218344]
16
[1.75753313]
17
[1.72098257]
18
[1.4615031]
19
[1.0307873]
20
[0.67728381]
21
[0.51993562]
22
[0.44033155]
23
[0.3317252]
0
[0.22654118]
Done processing NOX maps


In [2]:
from matplotlib import animation
from IPython.display import HTML

times = sorted(nox_local.keys())

fig, ax = plt.subplots(figsize=(8,8))
all_values = np.concatenate([nox_local[t].ravel() for t in times])
vmin, vmax = np.percentile(all_values, [2, 98])  # robust color scale


im = ax.imshow(nox_local[times[0]], cmap='jet', animated=True, vmin=0, vmax=100)
cbar = fig.colorbar(im, ax=ax)
title = ax.set_title(f"NOX  (traffic contribution) [ug/m3] at {times[0]}")
ax.axis("off")

# Update function for animation
def update(frame):
    data = nox_local[times[frame]]
    im.set_array(data)
    title.set_text(f"NOX  (traffic contribution) [ug/m3] at {times[0]}")
    return [im, title]

# Build the animation
ani = animation.FuncAnimation(
    fig, update, frames=len(times), interval=800, blit=True
)

# Show in Jupyter
HTML(ani.to_jshtml())


NameError: name 'nox_local' is not defined